In [ ]:
#| default_exp indexer

In [2]:
#| export
import ffmpeg
import numpy as np
import pandas as pd

from PIL import Image
from shutil import copy
from pathlib import Path

from projetMAB1.storage import Storage
from projetMAB1.model import Model

ModuleNotFoundError: No module named 'projetMAB1'

In [ ]:
#|export
def index(video_path: str, threshold = 0.10):

    video_name = Path(video_path).name

    if Path(video_path) != Path("../data/runtime/videos/" + video_name):
        copy(video_path, "../data/runtime/videos/" + video_name)

    out, _ = (
        ffmpeg.input(video_path, loglevel="warning")
        .filter('scale', size="film")
        .output('pipe:', format='rawvideo', pix_fmt='rgb24')
        .run(capture_stdout=True)
    )

    film = 352, 240
    width, height = film

    images = (
        np.frombuffer(out, np.uint8)
        .reshape([-1, height, width, 3])
    )

    df = Storage.load()

    empty_df = True if "video_name" not in df.columns else False

    model = Model()
    image_features = model.get_image_features(images)

    slices = [0]
    threshold = 0.15
    for i in range(1, len(images)):
        if 1 - image_features[i-1] @ image_features[i] >= threshold:
            slices.append(i)
    
    slices.append(len(images)+1)

    entries = []
    video_name = Path(video_path).stem
    for i in range(len(slices)-1):
        start = slices[i]
        end = slices[i+1]
        preview_name = f"{video_name}_{start}_{end}.png"
        entries.append((video_name, preview_name, start, end))

        Image.fromarray(images[slices[i]]).save("../data/runtime/frames/" + preview_name)

    new_df = pd.DataFrame(data = entries, columns= ["video_name", "preview_name", "start", "end"])

    if empty_df:
        df = new_df
    else:
        df = pd.concat([df, new_df], ignore_index=True)

    df.to_pickle("../data/runtime/index.pkl")

In [ ]:
df = pd.read_pickle("../data/runtime/index.pkl")
df = df.drop(df.index)
df.to_pickle("../data/runtime/index.pkl")

In [ ]:
videos = sorted(Path("../data/runtime/videos/").glob("*.mp4"))
for video in videos:
    index(str(video))

In [ ]:
pd.read_pickle("../data/runtime/index.pkl")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()